In [1]:
from __future__ import division
from astropy.io import fits
import os,glob,sys
import glob
import copy
import numpy as np
from astropy.table import Table, Column
import numpy as np
from sklearn import preprocessing

In [61]:
# define path
path = '/Users/runquanguan/Documents/136P2/Mix'

In [62]:
os.chdir(path)
filename = glob.glob('*')

In [4]:
def get_bias(folder):
    Bias = []
    ARRAY = []
    # pick out bias frame
    for file in folder:
        hdul = fits.open(file)
        TYPE = hdul[0].header['OBSTYPE']
        EXPT = hdul[0].header['EXPTIME']
        if TYPE == 'DARK' and EXPT==0:
            Bias.append(file)
        else:
            pass
            
    # extract data array from data 
    for file in Bias:
        hdul_b = fits.open(file)
        array = hdul_b[0].data
        # form arrays as a list
        ARRAY.append(array[0:950,0:1000])
    # turn list into array again to sum up
    ARRAY = np.array(ARRAY)
    # sum up
    SUM = ARRAY.sum(axis=0)
    # average out
    AVG = SUM/len(ARRAY)
    # return result
    return AVG


def get_dark(folder):
    dark = []
    ARRAY = []
    bias_frame = get_bias(folder)
    
    # pick dark
    for file in folder:
        hdul = fits.open(file)
        TYPE = hdul[0].header['OBSTYPE']
        EXPT = hdul[0].header['EXPTIME']
        FILTER = hdul[0].header['FILTNAM']
        if TYPE == 'DARK' and EXPT!=0:
            dark.append(file)
        else:
            pass
        
    # dark = dark-bias/expt
    for file in dark:
        hdul = fits.open(file)
        array = hdul[0].data
        # form arrays as a list
        ARRAY.append(array[0:950,0:1000])
    # turn list into array again to sum up
    ARRAY = np.array(ARRAY)
    # sum up
    SUM = ARRAY.sum(axis=0)
    # average out
    AVG = SUM/len(ARRAY)
    # return result
    return (AVG-bias_frame)/30


In [49]:
def B_Flat(folder):
    
    L = []
    ARRAY = []
    bias_frame = get_bias(folder)
    dark_frame = get_dark(folder)

    for file in filename:
        hdul = fits.open(file)
        header = hdul[0].header
        TYPE = hdul[0].header['OBSTYPE']
        FILTER = hdul[0].header['FILTNAM']
        FRAME = hdul[0].header['OBJECT']  
        EXPT = hdul[0].header['EXPTIME']
        if TYPE == 'OBJECT' and FILTER=='B' and FRAME=='dflat':
            L.append(file)
    # Flat = (file-dark-bias)/EXPT
    for file in L:
        hdul = fits.open(file)
        array = hdul[0].data
        ARRAY.append(array[0:950,0:1000])
        
    ARRAY = np.array(ARRAY)
    SUM = ARRAY.sum(axis=0)
    AVG = SUM/len(ARRAY)
    FLAT = (AVG - dark_frame*30 - bias_frame)/30
    FLAT = FLAT/np.mean(FLAT)
    return FLAT


def V_Flat(folder):
    
    L = []
    ARRAY = []
    bias_frame = get_bias(folder)
    dark_frame = get_dark(folder)

    for file in filename:
        hdul = fits.open(file)
        header = hdul[0].header
        TYPE = hdul[0].header['OBSTYPE']
        FILTER = hdul[0].header['FILTNAM']
        FRAME = hdul[0].header['OBJECT']  
        EXPT = hdul[0].header['EXPTIME']
        if TYPE == 'OBJECT' and FILTER=='V' and FRAME=='dflat':
            L.append(file)
    # Flat = (file-dark-bias)/EXPT
    for file in L:
        hdul = fits.open(file)
        array = hdul[0].data
        ARRAY.append(array[0:950,0:1000])
        
    ARRAY = np.array(ARRAY)
    SUM = ARRAY.sum(axis=0)
    AVG = SUM/len(ARRAY)
    FLAT = (AVG - dark_frame*3 - bias_frame)/30
    FLAT = FLAT/np.mean(FLAT)
    return FLAT

In [63]:
def reduce_B_frame(folder):
    
    L = []
    B_Reduced = []
    
    # define bias frame
    bias_frame = get_bias(folder)
    dark_frame = get_dark(folder)
    BFlat = B_Flat(folder)
    
    # find B science frame
    for file in filename:
        hdul = fits.open(file)
        header = hdul[0].header
        TYPE = hdul[0].header['OBSTYPE']
        FILTER = hdul[0].header['FILTNAM']
        FRAME = hdul[0].header['OBJECT']  
        EXPT = hdul[0].header['EXPTIME']
        if TYPE == 'OBJECT' and FILTER=='B' and FRAME=='M53_B':
            L.append(file)
        else:
            pass
             
    for file in L:
        hdul = fits.open(file)
        image_data = hdul[0].data[0:950,0:1000]
        
        image_header = hdul[0].header
        DATA = (image_data - bias_frame - dark_frame*EXPT) / BFlat
        new_file = file[0:4]+"_B_RDU"+file[4:8]
        outhdu = fits.PrimaryHDU(data = DATA, header = image_header)
        outhdu.writeto(new_file,overwrite=True)
        B_Reduced.append(new_file)
    return B_Reduced

def reduce_V_frame(folder):
    
    L = []
    V_Reduced = []
    
    # define bias frame
    bias_frame = get_bias(folder)
    dark_frame = get_dark(folder)
    VFlat = V_Flat(folder)
    
    # find B science frame
    for file in filename:
        hdul = fits.open(file)
        header = hdul[0].header
        TYPE = hdul[0].header['OBSTYPE']
        FILTER = hdul[0].header['FILTNAM']
        FRAME = hdul[0].header['OBJECT']  
        EXPT = hdul[0].header['EXPTIME']
        if TYPE == 'OBJECT' and FILTER=='V' and FRAME=='M53_V':
            L.append(file)
        else:
            pass
             
    for file in L:
        hdul = fits.open(file)
        image_data = hdul[0].data[0:950,0:1000]
        
        image_header = hdul[0].header
        DATA = (image_data - bias_frame - dark_frame*EXPT) / VFlat
        new_file = file[0:4]+"_V_RDU"+file[4:8]
        outhdu = fits.PrimaryHDU(data = DATA, header = image_header)
        outhdu.writeto(new_file,overwrite=True)
        V_Reduced.append(new_file)
    return V_Reduced


def reduce_R_frame(folder):
    
    L = []
    R_Reduced = []
    
    # define bias frame
    bias_frame = get_bias(folder)
    dark_frame = get_dark(folder)
    VFlat = V_Flat(folder)
    
    # find B science frame
    for file in filename:
        hdul = fits.open(file)
        header = hdul[0].header
        TYPE = hdul[0].header['OBSTYPE']
        FILTER = hdul[0].header['FILTNAM']
        FRAME = hdul[0].header['OBJECT']  
        EXPT = hdul[0].header['EXPTIME']
        if TYPE == 'OBJECT' and FILTER=='R' and FRAME=='M53_R':
            L.append(file)
        else:
            pass
             
    for file in L:
        hdul = fits.open(file)
        image_data = hdul[0].data[0:950,0:1000]
        
        image_header = hdul[0].header
        DATA = (image_data - bias_frame - dark_frame*EXPT) / VFlat
        new_file = file[0:4]+"_R_RDU"+file[4:8]
        outhdu = fits.PrimaryHDU(data = DATA, header = image_header)
        outhdu.writeto(new_file,overwrite=True)
        R_Reduced.append(new_file)
    return R_Reduced


In [64]:
reduce_R_frame(filename)

/Users/runquanguan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:57: RuntimeWarning: invalid value encountered in double_scalars


['d164_R_RDU.fit',
 'd168_R_RDU.fit',
 'd165_R_RDU.fit',
 'd166_R_RDU.fit',
 'd167_R_RDU.fit']